In [7]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.decomposition import NMF 

from scipy.sparse import csr_matrix

import pickle

In [8]:
R_df = pd.read_csv("./data/movie_matrix.csv", index_col=0)
movies_title_df = pd.read_csv("./data/movie_title.csv", index_col=0)
movies_title_df


,title
0,Toy Story (1995)
2,Grumpier Old Men (1995)
5,Heat (1995)
43,Seven (a.k.a. Se7en) (1995)
46,"Usual Suspects, The (1995)"
...,...
1435,Terms of Endearment (1983)
2968,Little Nicky (2000)
3158,Joe Dirt (2001)
2249,RoboCop 2 (1990)


In [9]:
with open('nmf_model1.pkl','rb') as file:
    loaded_model = pickle.load(file)

In [10]:
def recommend_nmf(query, model, k=10):
    """
    Filters and recommends the top k movies for any given input query based on a trained NMF model. 
    Returns a list of k movie ids.
    """
    
    # 1. construct new_user-item dataframe given the query
    new_user_df =  pd.DataFrame(query, columns=movies_title_df["title"], index=["new_user"]).fillna(0)

    # 2. scoring
    
        # calculate the score with the NMF model
    Q_matrix = model.components_
    Q = pd.DataFrame(Q_matrix)
    P_new_user_matrix = loaded_model.transform(new_user_df)
    R_hat_new_user_matrix = np.dot(P_new_user_matrix, Q)
    R_hat_new_user = pd.DataFrame(data=R_hat_new_user_matrix,
                         columns=movies_title_df["title"],
                         index = ['new_user'])

    # 3. ranking
    
        # filter out movies already seen by the user
    R_hat_new_user_filtered=R_hat_new_user.drop(query.keys(), axis=1)
    ranked = R_hat_new_user_filtered.T.sort_values(by=["new_user"], ascending=False).index.to_list()

        # return the top-k highest rated movie ids or titles
    recommendation = ranked[:3]
    return recommendation

In [11]:
query = {"Toy Story (1995)": 5,
                 "Grumpier Old Men (1995)":2,
                 "Heat (1995)":3.5}

In [12]:
recommend_nmf(query, loaded_model, k=10)

violation: 1.0
violation: 2.437198153353398
violation: 1.4315237694339609
violation: 0.21879880887531608
violation: 0.1556572184061561
violation: 0.06178911069071979
violation: 0.016458718854186236
violation: 0.009224578270520093
violation: 0.00233012395092348
violation: 0.0009454354827115188
violation: 0.0007042513193686026
violation: 0.00018168095090319352
violation: 2.6945226310217965e-05
Converged at iteration 14


['Rock, The (1996)', 'Trainspotting (1996)', 'Broken Arrow (1996)']